# Running NetPyNE in a Jupyter Notebook

## Preliminaries

We don't want to affect your system in any way, so we will operate from a virtual environment.  These preliminary steps must be completed before going through this tutorial.  You can't complete the preliminary steps from within Jupyter because you can't enter a virtual environment in Jupyter, from Jupyter you have to switch to a kernel made from your virtual environment.

First we will create and activate a virtual environment, then we will update pip and install iPython in the virtual environment, and finally we will create a kernel from the virtual environment that can be used by Jupyter.  

### Create and activate a virtual environment

First, open a Terminal and switch to the directory where you downloaded this notebook:

    cd Desktop/netpyne_tut1
    
Next, create a virtual environment named "env":

    python3 -m venv env
    
Check to see where you are currently running Python from:

    which python3
    
Enter your new virtual environment:

    source env/bin/activate
    
You should see in your prompt that you are in "env".  

Now see where you are running Python from:

    which python3
    
It should come from inside your new virtual environment.  Any changes we make here will only exist in the "env" directory that was created here.  

To exit your virtual environment, enter:

    deactivate
    
Your prompt should reflect the change.  To get back in, enter:

    source env/bin/activate
    
### Update pip and install iPython

We will now update pip and install iPython in the virtual environment.  From inside your virtual environment, enter:

    python3 -m pip install --upgrade pip
    python3 -m pip install --upgrade ipython
    
### Make a Jupyter kernel out of this virtual environment

Now we will create a kernel that can be used by Jupyter Notebooks.  Enter:

    ipython kernel install --user --name=env

### Launch this notebook in Jupyter Notebook

Now we will launch Jupyter from within the virtual environment.  Enter:

    jupyter notebook netpyne_tut1.ipynb
    
This should open a web browser with Jupyter running this notebook.  From the menu bar, click on "Kernel", hover over "Change kernel" and select "env".  We are now operating in the virtual environment (see "env" in the upper right) and can install our software.

### To run this again in the future

Be sure to enter your virtual environment before running Jupyter!

    cd Desktop/netpyne_tut1
    source env/bin/activate
    jupyter notebook netpyne_tut1.ipynb
    
And then make sure you are in the **env** kernel in Jupyter.

## Software installation

Once you are in the **env** kernel in Jupyter, you can execute the following commands to install the software.  The exclamation point at the beginning means this is a command to be executed in your terminal (in your current virtual environment / kernel).  You can also execute these commands directly in your terminal, just leave off the exclamation point.

First we will install NEURON.  Shift+Enter the following:

In [1]:
!python3 -m pip install --upgrade neuron

Requirement already up-to-date: neuron in /Users/salvadord/temp/env/lib/python3.8/site-packages (7.8.1.1)
You should consider upgrading via the 'pip install --upgrade pip' command.


Now we will install NetPyNE.  Shift+Enter the following:

In [2]:
!python3 -m pip install --upgrade netpyne

  Using cached https://files.pythonhosted.org/packages/55/87/e6688fc9d9f298523d2bdc1a39d421768fc4c7a075efc4d9d016297a080b/netpyne-0.9.8-py2.py3-none-any.whl
  Found existing installation: netpyne 0.9.5
    Not uninstalling netpyne at /Users/salvadord/Documents/ISB/Models/netpyne_repo, outside environment /Users/salvadord/temp/env
    Can't uninstall 'netpyne'. No files were found to uninstall.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Tutorial 1 -- a simple network with one population

Now we are ready to start NetPyNE Tutorial 1, which will create a simple network model we can simulate.  We will create a fairly simple network model of 40 pyramidal-like, two-compartment neurons with standard Hodgkin-Huxley dynamics in the somas and passive dynamics in the dendrites.  We will then connect the neurons randomly with a 10% probability of connection using a standard double-exponential synapse model.  Finally, we will add a current clamp stimulus to one cell to activate the network.  Then we will explore the model.

## Instantiate network parameters and simulation configuration

You need two things to define a model/simulation in NetPyNE: 1) the parameters of the network and all its components and 2) the configuration of the simulation.  These requirements exist as objects in NetPyNE.  Let's instantiate them now.

In [4]:
from netpyne import specs, sim
netParams = specs.NetParams()
simConfig = specs.SimConfig()

ModuleNotFoundError: No module named 'neuron'

The following command plots all figures directly in this notebook:

In [ ]:
%matplotlib inline

These NetPyNE objects come with a lot of defaults set which you can explore with tab completion, but we'll focus on that more later.

We are going to plunge ahead and build our model: a simple network of 40 pyramidal-like two-compartment neurons with standard Hodgkin-Huxley dynamics in the soma and passive dynamics in the dendrite.  

## Create a cell model

First we will add a cell type to our model by adding a dictionary named **pyr** to the *Cell Parameters* dictionary (**cellParams**) in the *Network Parameters* dictionary (**netParams**).  We will then add an empty dictionary named **secs** to hold our compartments.

In [ ]:
netParams.cellParams['pyr'] = {}
netParams.cellParams['pyr']['secs'] = {}

### Specify the soma compartment properties

Now we will define our **soma**, by adding a **geom** dictionary defining the geometry of the soma and a **mechs** dictionary defining the biophysical mechanics being added to the soma.

In [ ]:
netParams.cellParams['pyr']['secs']['soma'] = {}

In [ ]:
netParams.cellParams['pyr']['secs']['soma']['geom'] = {
    "diam": 12,
    "L": 12,
    "Ra": 100.0,
    "cm": 1
    }

In [ ]:
netParams.cellParams['pyr']['secs']['soma']['mechs'] = {"hh": {
    "gnabar": 0.12,
    "gkbar": 0.036,
    "gl": 0.0003,
    "el": -54.3
    }}

### Specify the dendrite compartment properties

Next will do the same thing for the dendrite compartment, but we will do it slightly differently.  We will first build up a **dend** dictionary and then add it to the cell model dictionary **pyr** when we are done.

In [ ]:
dend = {'geom': {}, 'mechs': {}, 'topol': {}}

In [ ]:
dend['geom'] = {"diam": 1.0,
                "L": 200.0,
                "Ra": 100.0,
                "cm": 1,
               }

In [ ]:
dend['mechs'] = {"pas": 
                    {"g": 0.001,
                     "e": -70}
                }

In order to connect the dendrite compartment to the soma compartment, we must add a **topol** dictionary to our **dend** dictionary.

In [ ]:
dend['topol'] = {"parentSec": "soma",
                 "parentX": 1.0,
                 "childX": 0,
                }

With our **dend** section dictionary complete, we must now add it to the **pyr** cell dictionary.

In [ ]:
netParams.cellParams['pyr']['secs']['dend'] = dend

Our two-compartment cell model is now completely specified.  Note that the *hh* and *pas* mechanisms we specified above are builtin to NEURON.  Our next step is to create a population of these cells.

## Create a population of cells

NetPyNE uses *populations* of cells to specify connectivity.  In this tutorial, we will create just one population which we will call **E** (for excitatory).  It will be made of the **pyr** cells we just specified, and we want 40 of them.

In [ ]:
netParams.popParams['E'] = {
    "cellType": "pyr",
    "numCells": 40,
}

## Create a synaptic model

We need a synaptic mechanism to connect our cells with.  We will create one called **exc** by adding a dictionary to the *synaptic mechanism parameters* dictionary (**synMechParams**).  The synapse *mod* used (**Exp2Syn**) is a simple double-exponential which is builtin to NEURON.

In [ ]:
netParams.synMechParams['exc'] = {
    "mod": "Exp2Syn",
    "tau1": 0.1,
    "tau2": 1.0,
    "e": 0
}

## Connect the cells

Now we will specify the connectivity in our model by adding a dictionary to the **connParams** dictionary.  We will call our connectivity rule **E->E** as it will define connectivity from our **E** population to our **E** population.

We will use the *synMech* **exc**, which we defined above.  For this synaptic mechanism, a *weight* of about **0.005** is appropriate.  These cells will have a 10% probability of getting connected, and will be activated five milliseconds after an action potential occurs in the presynaptic cell.  Synapses will occur on the **dend** *section* at its very end (*location* **1.0**)

In [ ]:
netParams.connParams['E->E'] = {
    "preConds": {
        "pop": "E",
    },
    "postConds": {
        "pop": "E",
    },
    "weight": 0.005,
    "probability": 0.1,
    "delay": 5.0,
    "synMech": "exc",
    "sec": "dend",
    "loc": 1.0,
}

## Set up the simulation configuration

In [ ]:
simConfig.duration = 200.0
simConfig.dt = 0.1
simConfig.recordCells = [0]
simConfig.recordTraces = {
    "V_soma": {
        "sec": "soma",
        "loc": 0.5,
        "var": "v",
    },
    "V_dend": {
        "sec": "dend",
        "loc": 1.0,
        "var": "v",
    }
}
simConfig.filename = "netpyne_tut1"
simConfig.analysis = {
    "plotTraces": {
        "include": [0],
        "overlay": True,
        "showFig": False,
        "saveFig": True,
    },
    "plotRaster": {
        "markerSize": 5,
        "showFig": False,
        "saveFig": True,
    }
}

## Create, simulate, and analyze the model

Use one simple command to create, simulate, and analyze the model.

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

### Plot the 2D connectivity of the network

In [ ]:
fig, figData = sim.analysis.plot2Dnet()

### Plot the connectivity matrix

In [ ]:
fig, figData = sim.analysis.plotConn()

## Add a stimulation

We'll need to kickstart this network -- let's inject current into one of the cells.  First we need to add a dictionary to the *Stimulation Source Parameters* dictionary (**stimSourceParams**).  We'll call our stimulation **IClamp1**, and we'll use the standard NEURON *type*: **IClamp**.  The current injection will last for a *duration* of 20 ms, it will start at a *delay* of 5 ms, and it will have an *amplitude* of 0.1 nanoAmps. 

In [ ]:
netParams.stimSourceParams['IClamp1'] = {
    "type": "IClamp",
    "dur": 5,
    "del": 20,
    "amp": 0.1,
}

Now we need to add a target for our stimulation.  We do that by adding a dictionary to the *Stimulation Target Parameters* dictionary (**stimTargetParams**).  We'll call this connectivity rule **IClamp1->cell0**, because it will go from the source we just created (**IClamp1**) and the first cell in our population.  The stimulation (current injection in this case) will occur in our **dend** *section* at the very tip (*location* of **1.0**).

In [ ]:
netParams.stimTargetParams['IClamp1->cell0'] = {
    "source": "IClamp1",
    "conds": {
        "cellList": [0]
    },
    "sec": "dend",
    "loc": 1.0,
}

## Create, simulate, and analyze the model


In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

### Plot the connectivity matrix

In [ ]:
fig, figData = sim.analysis.plotConn()

## Record and plot a variety of traces

### Record and plot the somatic conductances

Let's record and plot the somatic conductances.  
HH mod is here: https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/hh.mod
Passive mod is here: https://github.com/neuronsimulator/nrn/blob/master/src/nrnoc/passive.mod

In [ ]:
simConfig.recordTraces['gNa'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gna'}

In [ ]:
simConfig.recordTraces['gK'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gk'}

In [ ]:
simConfig.recordTraces['gL'] = {'sec': 'soma', 'loc': 0.5, 'mech': 'hh', 'var': 'gl'}

In [ ]:
del simConfig.recordTraces['V_soma']

In [ ]:
del simConfig.recordTraces['V_dend']

In [ ]:
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

In [ ]:
simConfig.recordTraces

### Record all somatic currents

### Record from a synapse

### Record ionic concentratins

In [ ]:
#cfg.recordTraces = {'ca': {'sec': 'soma', 'loc': 0.5, 'var': 'cai'},

## Access and manipulate data

In [5]:
!which nrnivmodl

/Users/salvadord/temp/env/bin/nrnivmodl
